# Exercício Programa 1
# Nome: Lucas Martins Marques
# NUSP: 12566592

In [8]:
from heapq import heappush, heappop

class Node:
    def __init__(self, car=None,freq=None,left=None,right=None):
        self.caracter = car
        self.frequencia = freq
        self.left = left
        self.right = right  
        if (self.caracter is None): #Utilizado na funcao preordertraverse
            self.str = '0'
        else:
            self.str = '1'

    def __lt__(self,other):
        if (self.frequencia <= other.frequencia):
            return(True)
        else:
            return(False)
            
class BST:
    def __init__(self,root=None):
        self.root = root
        
    def devolveroot(self):
        return self.root
    
    # 6) Construcao da tabela de codificacao onde cada linha se refere a uma caracter e o seu codigo de compactacao
    def lista_de_codigos(self, p=None):
        codigos = []
        codigo = ""
        def recursiva_lista_de_codigos(p):
            nonlocal codigo
            if(p is not None):
                if (p.caracter is not None):
                    codigos.append((p.caracter, codigo))
                    codigo = codigo[0:-1]
                else:
                    codigo += "0"
                    recursiva_lista_de_codigos(p.left)
                    codigo += "1"
                    recursiva_lista_de_codigos(p.right)
                    codigo = codigo[0:-1]
        recursiva_lista_de_codigos(p)
        return codigos

    # 8) Geracao da arvore escrita sob a forma linear
    def preordertraverse(self,p):
        arvore_linear = []
        def recursiva_preordertraverse(p):
            nonlocal arvore_linear
            if(p is not None):
                if (p.caracter is not None):
                    arvore_linear.append(p.str)
                    caracter_binario = format(ord(p.caracter), 'b')
                    arvore_linear.append(caracter_binario.zfill(8))
                else:
                    arvore_linear.append(p.str)
                    recursiva_preordertraverse(p.left)
                    recursiva_preordertraverse(p.right)
        recursiva_preordertraverse(p)
        return arvore_linear

def list_to_str(lista):
    string = ""
    for i in lista:
        string += i
    return string

def completa_byte(s, lado):
    l = list(s)
    while len(l) % 8 != 0 and len(l) != 0:
        if lado == "direita":
            l.append("0")
        elif lado == "esquerda":
            l.insert(0,"0")
        else:
            raise Exception('o parametro lado aceita apenas "esquerda" ou "direita"')
    s = list_to_str(l)
    return s

def arv_linear_para_arv(arvore_linear):
    arvore_linear = arvore_linear[1:]

    def arv_linear_para_arv_left():
        nonlocal arvore_linear
        if arvore_linear[0] == "1":
            car = chr(int(arvore_linear[1:9], 2))
            left = Node(car)
            arvore_linear = arvore_linear[9:]
        else:
            arvore_linear = arvore_linear[1:]
            left = Node(None,None, arv_linear_para_arv_left(), arv_linear_para_arv_right())
        return left

    def arv_linear_para_arv_right():
        nonlocal arvore_linear
        if arvore_linear[0] == "1":
            car = chr(int(arvore_linear[1:9], 2))
            right = Node(car)
            arvore_linear = arvore_linear[9:]
        else:
            arvore_linear = arvore_linear[1:]
            right = Node(None, None , arv_linear_para_arv_left(), arv_linear_para_arv_right())
        return right

    raiz = Node(None, None, arv_linear_para_arv_left(), arv_linear_para_arv_right())
    return BST(raiz)
        
def codificacao(arquivo_original):
    # 1) e 2) Leitura do arquivo e Geracao de estatistica
    arq = list(arquivo_original)
    estatistica = []
    while arq:
        freq = 0
        caracter = arq[0]
        j = 0
        while j < len(arq):
            if arq[j] == caracter:
                freq += 1
                del arq[j]
            else:
                j += 1
        estatistica.append((caracter, freq))

    # 3) e 4) Criacao de nos para cada um dos caracteres e Insercao dos nos numa fila de prioridades
    prioridades = []
    for i in estatistica:
        node = Node(i[0], i[1])
        heappush(prioridades, (node.frequencia, node))

    # 5) Criacao da arvore de binaria que contem as informacoes de codificacao
    while len(prioridades) > 1:
        a = heappop(prioridades)
        b = heappop(prioridades)
        c = Node(None, a[0] + b[0], a[1], b[1])
        heappush(prioridades, (c.frequencia, c))
    arvore = BST(prioridades[0][1])

    # 6) Construcao da tabela de codificacao onde cada linha se refere a uma caracter e o seu codigo de compactacao
    codigos = arvore.lista_de_codigos(prioridades[0][1])
    nc = str(len(codigos)).zfill(3)

    # 7) Geracao dos bytes que representam a informacao compactada
    arq = list(arquivo_original)
    for i in range(len(arq)):
        for j in range(len(codigos)):
            if arq[i] == codigos[j][0]:
                arq[i] = codigos[j][1]
    arq = list_to_str(arq)
    nb = str(len(arq)).zfill(4)

    # 8) Geracao da arvore escrita sob a forma linear
    arvore_linear = list_to_str(arvore.preordertraverse(prioridades[0][1]))
    
    # 9) Geracao do arquivo compactado
    bytes = []
    #bytes nc
    for i in nc:
        bytes.append(ord(i))
    #bytes nb
    for i in nb:
        bytes.append(ord(i))
    #bytes restantes
    bytes_finais_str = completa_byte(arvore_linear + arq, "direita")
    while bytes_finais_str:
        bytes.append(int(bytes_finais_str[:8], 2))
        bytes_finais_str = bytes_finais_str[8:]
    #escreve no arquivo
    with open("codigo.huf", "wb") as arquivo_final:
        arquivo_final.write(bytearray(bytes))

def decodificacao(arquivo_original):
    #Leitura do arquivo
    cadeia = []
    try:
        with open(arquivo_original, "rb") as arq:
            flag = True
            while flag:
                byte = arq.read(1)
                if byte:
                    cadeia.append(int.from_bytes(byte, "big"))
                else:
                    flag = False
    except:
        print("Erro ao abrir o arquivo")
        return
            
    #Particao do arquivo de acordo com as especificacoes
    nc, cadeia = int(chr(cadeia[0]) + chr(cadeia[1]) + chr(cadeia[2])), cadeia[3:]
    nb, cadeia = int(chr(cadeia[0]) + chr(cadeia[1]) + chr(cadeia[2]) + chr(cadeia[3])), cadeia[4:]

    #Leitura da parte final do arquivo
    binario = ""
    get_bin = lambda x: format(x, 'b').zfill(8)
    for i in cadeia:
        binario += get_bin(i)

    #separa binario em arvore linearizada e conteudo codificado
    i = 0
    while nc > 0 and i < len(binario):
        if binario[i] == "1":
            i += 9
            nc -= 1
        else:
            i += 1
    arvore_linear = binario[:i]
    binario = binario[i:]
    info_codificada = binario[:nb]

    #cria a arvore
    arvore = arv_linear_para_arv(arvore_linear)

    #cria lista de codigos
    p = arvore.devolveroot()
    codigos = arvore.lista_de_codigos(p)

    #decodifica a info codificada
    string = ""
    with open("arquivo_final.txt", "w", encoding = 'utf-8') as arquivo:
        for i in info_codificada:
            string += i
            for j in codigos:
                if string == j[1]:
                    car = j[0]
                    arquivo.write(car)
                    string = ""
    
    print('Sua descompactacao esta salva no arquivo "arquivo_final.txt"')
        
def main():
    operacao = input("Selecione a operação que deseja\nPara codificação digite 0\nPara decodificação digite 1\n")
    if operacao == "0":
        arq_ou_txt = input("Digite 0 para inserir o nome de um arquivo ou 1 para inserir um texto: ")
        if arq_ou_txt == "0":
            try:
                arquivo = input("Digite o nome do arquivo: ")
                with open(arquivo, "r", encoding = 'utf-8') as arq:
                    arquivo = arq.read()
            except FileNotFoundError:
                print("\nO arquivo nao existe!\n")
                return
        elif arq_ou_txt == "1":
            arquivo = input("Digite o texto: ")
        else:
            print("\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print("O valor deve ser 0 ou 1\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n")
            return
        codificacao(arquivo)
        print('Seu arquivo arquivo compactado foi salvo no arquivo "codigo.huf"')
    elif operacao == "1":
        arquivo = input("Digite o nome do arquivo: ")
        decodificacao(arquivo)
    else:
        print("\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("O valor deve ser 0 ou 1\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n")

if __name__ == "__main__": main()

Selecione a operação que deseja
Para codificação digite 0
Para decodificação digite 1
1
Digite o nome do arquivo: codigo.huf
Sua descompactacao esta salva no arquivo "arquivo_final.txt"


## Resumo do cógido 

   O primeiro passo do programa é a função main(). Esta função dá ao usuário a possibilidade de escolher se quer utilizar a função codificacao() ou decodificacao(). 

   O usuário, pela função main(), escolhe se quer inserir um texto ou um arquivo, quando seleciona a função codificação(). Ao receber este texto ou arquivo a função segue um passo a passo: 1) lê o arquivo e gera estatísticas, uma lista com tuplas dos caracteres e suas respectivas frequências; 2) cria nós para cada um dos caracteres com a classe Node e os insere em um fila de prioridades manejada por uma heap, usando a biblioteca heapq; 3) cria uma árvore binária que contém as informações de codificação, por meio do mecanismo de codificação de Huffman, que minimiza o número de bits do arquivo final codificado, e a árvore é criada com a classe BST; 4) constrói a tabela de codificação onde cada linha se refere a um caractere e seu código de codificação, usando a função recursiva lista_de_codigos() da classe BST, além de criar a variável nc contendo o número de caracteres distintos; 5) gera os bytes que representam a informação compactada com 0s e 1s e cria a variável nb que contém o número de bits da informação codificada; 6) gera a árvore escrita sob a forma linear com a função recursiva preordertraverse() da classe BST, que classifica com “1” seguido do código ASCII do caractere do nó terminal e 0 caso seja um nó não terminal; 7) gera o arquivo compactado, sendo os três primeiros bytes são uma string com nc, os quatro próximos também é uma string com nb, os próximos são uma sequência de 0s e 1s com a árvore linearizada e por fim são os bytes com a informação codificada. O arquivo compactado é criado com o nome “código.huf” na mesma pasta.
  
   Ao selecionar decodificacao(), o usuário passa o nome do arquivo codificado. Ao receber o arquivo a função 1) lê o arquivo, transformando em uma lista de inteiros; 2) recolhe os números de caracteres e de bits da informação codificada; 3) lê o resto do arquivo, transformando em 0s e 1s; 4) separa o binário em árvore linearizada e conteúdo codificado; 5) recria a árvore pela árvore linearizada com a função arv_linear_para_arv(); 6) recria a lista de códigos com a função lista_de_codigos() da classe BST; 7) decodifica a informação e já escreve no arquivo final, salvo como “arquivo_final.txt” na mesma pasta.
   
   Os resultados obtidos pelos testes feitos, tanto com o arquivo “Alice1.txt”, como com textos aleatórios foram plenamente satisfatórios. Por mais desafiador que foi criar esse programa, foi extremamente recompensador ver que estava funcionando perfeitamente.
